# Importing Required Libraries and Packages

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import plotly.offline as of
import plotly.graph_objs as gro
import plotly.tools as tls
import plotly.figure_factory as ff

Import Churn Dataset

In [ ]:
import io
from google.colab import files
uploaded = files.upload()
dataset = pd.read_csv(io.BytesIO(uploaded['Churn.csv']))
print(dataset.shape)
dataset.head()

Saving Churn.csv to Churn (3).csv
(7043, 21)


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [ ]:
datasetCopy = pd.read_csv(io.BytesIO(uploaded['Churn.csv'])) #making a copy of dataset

# EDA of Dataset

In [ ]:
train_stats = dataset.describe()
train_stats.transpose
train_stats

,SeniorCitizen,tenure,MonthlyCharges
count,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692
std,0.368612,24.559481,30.090047
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.500000
50%,0.000000,29.000000,70.350000
75%,0.000000,55.000000,89.850000
max,1.000000,72.000000,118.750000


CustomerID doesn't affect Churn anyway so we remove the column from the dataset

In [ ]:
dataset.pop('customerID')
dataset.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


Converting Feature values to numerical categories(0, 1, 2, 3)
Here I used Column Transform Method of sklearn

In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OrdinalEncoder
categorical_features = ['gender', 'Partner', 'Dependents','PhoneService','MultipleLines','InternetService','OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies','Contract','PaperlessBilling','PaymentMethod','Churn']
column_transform = make_column_transformer((OrdinalEncoder(), categorical_features))

column_transformed = column_transform.fit_transform(dataset)

In [ ]:
data = dataset.copy()
data = pd.DataFrame(column_transformed, columns = categorical_features)
dataset.update(data)
print(dataset.shape)
dataset.head(20)

(7043, 20)


,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,0,0,1,0,1,0,1,0,0,2,0,0,0,0,0,1,2,29.85,29.85,0
1,1,0,0,0,34,1,0,0,2,0,2,0,0,0,1,0,3,56.95,1889.5,0
2,1,0,0,0,2,1,0,0,2,2,0,0,0,0,0,1,3,53.85,108.15,1
3,1,0,0,0,45,0,1,0,2,0,2,2,0,0,1,0,0,42.30,1840.75,0
4,0,0,0,0,2,1,0,1,0,0,0,0,0,0,0,1,2,70.70,151.65,1
5,0,0,0,0,8,1,2,1,0,0,2,0,2,2,0,1,2,99.65,820.5,1
6,1,0,0,1,22,1,2,1,0,2,0,0,2,0,0,1,1,89.10,1949.4,0
7,0,0,0,0,10,0,1,0,2,0,0,0,0,0,0,0,3,29.75,301.9,0
8,0,0,1,0,28,1,2,1,0,0,2,2,2,2,0,1,2,104.80,3046.05,1
9,1,0,0,1,62,1,0,0,2,2,0,0,0,0,1,0,0,56.15,3487.95,0


#Data Manipulation

Checking for missing values

In [ ]:
dataset.isna().sum()

gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

Replacing Spaces with NaN values

In [ ]:
dataset.replace(r'^\s*$', np.nan, regex=True)
dataset.isna().sum()

gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [ ]:
dataset.iloc[:, 18] = pd.to_numeric(dataset.iloc[:, 18], errors='coerce')

Here we found that there were 11 spaces in TotalCharges Column which have been replaced by NaN, now we can fill those NaN values with mean/median as there aren't outliers as per data statistics.

In [ ]:
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(missing_values = np.nan, strategy = 'mean')

In [ ]:
dataset.iloc[:, 18] = imp_mean.fit_transform(dataset.iloc[:, 18].values.reshape(-1,1))

In [ ]:
dataset.isna().sum()

gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [ ]:
dataset = dataset.astype({"Churn":'category'}) 
#Churn column was converted to numerical so now converted back to categorical column

In [ ]:
dataset

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,0,0,1,0,1,0,1,0,0,2,0,0,0,0,0,1,2,29.85,29.85,0.0
1,1,0,0,0,34,1,0,0,2,0,2,0,0,0,1,0,3,56.95,1889.50,0.0
2,1,0,0,0,2,1,0,0,2,2,0,0,0,0,0,1,3,53.85,108.15,1.0
3,1,0,0,0,45,0,1,0,2,0,2,2,0,0,1,0,0,42.30,1840.75,0.0
4,0,0,0,0,2,1,0,1,0,0,0,0,0,0,0,1,2,70.70,151.65,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,1,0,1,1,24,1,2,0,2,0,2,2,2,2,1,1,3,84.80,1990.50,0.0
7039,0,0,1,1,72,1,2,1,0,2,2,0,2,2,1,1,1,103.20,7362.90,0.0
7040,0,0,1,1,11,0,1,0,2,0,0,0,0,0,0,1,2,29.60,346.45,0.0
7041,1,1,1,0,4,1,2,1,0,0,0,0,0,0,0,1,3,74.40,306.60,1.0


# **Correlation**

In [ ]:
correlation = dataset.corr()
matrix_cols = correlation.columns.tolist()
corr_array  = np.array(correlation)


trace = gro.Heatmap(z = corr_array,
                   x = matrix_cols,
                   y = matrix_cols,
                   colorscale = "Inferno",
                   colorbar   = dict(title = "Pearson Correlation coefficient",
                                     titleside = "right"
                                    ) ,
                  )

layout = gro.Layout(dict(title = "Correlation Matrix for variables",
                        autosize = False,
                        height  = 1000,
                        width   = 1000,
                        margin  = dict(r = 0 ,l = 200,
                                       t = 25,b = 200,
                                      ),
                        yaxis   = dict(tickfont = dict(size = 10)),
                        xaxis   = dict(tickfont = dict(size = 10))
                       )
                  )

data = [trace]
fig = gro.Figure(data=data,layout=layout)
of.iplot(fig)

Saperating Independent and Dependent Variables

In [ ]:
Y = dataset['Churn']
X = dataset.drop('Churn', axis=1)

Deviding Train and Test Dataset

In [ ]:
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, train_size = 0.8, test_size=0.2, random_state=5)

# 5. Building Different Models and Comparision

LogisticRegression Model

In [ ]:
logmodel = LogisticRegression(max_iter = 1000)
logmodel.fit(X_train,Y_train)
log_prediction = logmodel.predict(X_test)
logmodel_accuracy = accuracy_score(Y_test, log_prediction) * 100

Support Vector Machine Models

In [ ]:
svcmodel = SVC(kernel='rbf', probability=True)
svcmodel.fit(X_train,Y_train)
svc_prediction = svcmodel.predict(X_test)
svc_accuracy = accuracy_score(Y_test, svc_prediction) * 100

In [ ]:
linearSVC = LinearSVC(C=5.0, dual=False, loss="squared_hinge", penalty="l2", tol=0.0001)
linearSVC.fit(X_train, Y_train)
linear_prediction = linearSVC.predict(X_test)
linear_accuracy = accuracy_score(Y_test, linear_prediction) * 100

RandomForest

In [ ]:
rf = RandomForestClassifier(criterion="entropy", n_estimators=200)
rf.fit(X_train, Y_train)
rf_prediction = rf.predict(X_test)
rf_accuracy = accuracy_score(Y_test, rf_prediction) * 100

In [ ]:
k-NearestNeighbor

In [ ]:
knnmodel = KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=2)
knnmodel.fit(X_train, Y_train) 
  
knn_prediction = knnmodel.predict(X_test)

knn_accuracy = accuracy_score(Y_test, knn_prediction) * 100

Compare Several models according to their Accuracies


In [ ]:
Model_Comparison = pd.DataFrame({
    'Model': ['Logistic Regression', 'Support Vector Machine -RBF', 'LinearSVC' ,  'Random Forest', 'K-Nearest Neighbor'],
    'Score': [logmodel_accuracy, svc_accuracy,  linear_accuracy,  rf_accuracy, knn_accuracy]})
Model_Comparison_df = Model_Comparison.sort_values(by='Score', ascending=False)
Model_Comparison_df = Model_Comparison_df.set_index('Score')
Model_Comparison_df.reset_index()

,Score,Model
0,79.914833,Logistic Regression
1,79.843861,LinearSVC
2,79.772889,Random Forest
3,76.295245,K-Nearest Neighbor
4,72.604684,Support Vector Machine -RBF


Generate confusion matrix for logistics regression model as it has maximum Accuracy


In [ ]:
from sklearn.metrics import confusion_matrix
conf_mat_logmodel = confusion_matrix(Y_test, log_prediction)
conf_mat_logmodel

array([[918, 105],
       [178, 208]])

In [ ]:
#Predict the probability of Churn of each customer
dataset['Probability_of_Churn'] = logmodel.predict_proba(dataset[X_test.columns])[:,1]

Now we have predicted the probabilty of Churn for each customer
Now Dataset will look like this.

In [ ]:
dataset.head(10)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,Probability_of_Churn
0,0,0,1,0,1,0,1,0,0,2,0,0,0,0,0,1,2,29.85,29.85,0.0,0.589153
1,1,0,0,0,34,1,0,0,2,0,2,0,0,0,1,0,3,56.95,1889.50,0.0,0.048483
2,1,0,0,0,2,1,0,0,2,2,0,0,0,0,0,1,3,53.85,108.15,1.0,0.303500
3,1,0,0,0,45,0,1,0,2,0,2,2,0,0,1,0,0,42.30,1840.75,0.0,0.025557
4,0,0,0,0,2,1,0,1,0,0,0,0,0,0,0,1,2,70.70,151.65,1.0,0.664982
5,0,0,0,0,8,1,2,1,0,0,2,0,2,2,0,1,2,99.65,820.50,1.0,0.779064
6,1,0,0,1,22,1,2,1,0,2,0,0,2,0,0,1,1,89.10,1949.40,0.0,0.534613
7,0,0,0,0,10,0,1,0,2,0,0,0,0,0,0,0,3,29.75,301.90,0.0,0.315734
8,0,0,1,0,28,1,2,1,0,0,2,2,2,2,0,1,2,104.80,3046.05,1.0,0.593612
9,1,0,0,1,62,1,0,0,2,2,0,0,0,0,1,0,0,56.15,3487.95,0.0,0.010205


Now we can find Correlation between Probability of Churn and Input Features and higher correlation means that feature will afftect the probabiltiy the more.

In [ ]:
correlation = dataset.corr()
matrix_cols = correlation.columns.tolist()
corr_array  = np.array(correlation)


trace = gro.Heatmap(z = corr_array,
                   x = matrix_cols,
                   y = matrix_cols,
                   colorscale = "Inferno",
                   colorbar   = dict(title = "Pearson Correlation coefficient",
                                     titleside = "right"
                                    ) ,
                  )

layout = gro.Layout(dict(title = "Correlation Matrix for variables",
                        autosize = False,
                        height  = 1000,
                        width   = 1000,
                        margin  = dict(r = 0 ,l = 200,
                                       t = 25,b = 200,
                                      ),
                        yaxis   = dict(tickfont = dict(size = 10)),
                        xaxis   = dict(tickfont = dict(size = 10))
                       )
                  )

data = [trace]
fig = gro.Figure(data=data,layout=layout)
of.iplot(fig)

# Conclusions

Corrlation tells that tenure and TotalCharges will affect the probabilty of Churn the most.

I didn't use Categorical Feature for the Calculation of Correlation as I just wanted to tell that I know the correlation affects multicolinearity and Output and I found that LinearSVC model worked very good but that was still below Logistic Regression Model.

Here predicting the Churn is more important than accuracy as that affect the company the most, so here we should assign class weight to 'Churn = yes', that will tell us more about the customers who are likely to leave the service.

Assuming suggestion were implemented then we can conclude that actions worked if there is more correlation between change in those factors wrt to change in Customer Churn.